2. 내 반경 5km 이내의 병원 마커

In [1]:
import folium
import pandas as pd
import json
from folium import plugins
import geocoder
from geopy.distance import great_circle

In [2]:
with open('./전국_정신건강관련기관_위치정보.json', 'r',encoding='utf-8') as f:
    loc_data = json.load(f)
    
loc_arr = loc_data['data']
loc_arr

df = pd.DataFrame(loc_arr)

df = df.rename(columns={'위도':'lat', '경도':'lng'})
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lng'] = pd.to_numeric(df['lng'], errors='coerce')


df.index = range(len(df))
df

,index,공공/민간,기관구분,기관명,주소,lat,lng,전화번호,홈페이지
0,0,공공,상담소,(주)마음꼭 차심리상담센터,경기도 성남시 분당구 야탑동 351 기억력센터 2층 (야탑로 59 분당차병원),37.410250,127.125250,031-780-5437,None
1,1,공공,상담소,(주)마음연구소 부설 마음iN심리상담센터,서울 종로구 종로3길 34 6층,37.571875,126.979481,02-722-2883,None
2,2,공공,상담소,가능성연구소,부산시 해운대구 세실로79 부산해운대대교사옥 6층,35.171647,129.176440,051-704-8610,None
3,3,공공,상담소,가톨릭대학교 은평성모병원,서울 은평구 진관동 93-6,37.633251,126.916547,02-1811-7755,None
4,4,공공,상담소,강원대학교병원 정신건강의학과,강원도 춘천시 백령로 156 강원대학교병원 정신건강의학과,37.874701,127.744801,033-258-2437,None
...,...,...,...,...,...,...,...,...,...
4327,4706,민간,병원,실버한방병원,서울특별시 금천구 독산로 362 (독산동),37.479089,126.904289,02-857-2001,None
4328,4707,민간,병원,의료법인 영명의료재단 혜민한방병원,"경기도 시흥시 군자로 526 도일그랜드프라자 302,401,501호 (거모동)",37.343354,126.785183,031-504-7272,None
4329,4708,민간,병원,한도한방병원,경기도 의정부시 입석로 36 (가능동),37.755883,127.021652,031-877-6081,None
4330,4709,민간,병원,혜민한방병원,충청남도 태안군 태안읍 정주내1길 13-0 혜민한방병원,36.745075,126.304532,041-400-7171,None


In [3]:
user_loc = geocoder.ip('me')

mylocation = user_loc.latlng
print(mylocation)

mymap = folium.Map(location=mylocation, zoom_start=14, width='70%', height='100%')
folium.Marker(location=mylocation, popup='현재 나의 위치', icon=folium.Icon(color='red', icon='star')).add_to(mymap)
plugins.LocateControl().add_to(mymap)
folium.Circle(
    location=mylocation,
    radius=2000,
    color='#eb9e34',
    fill_color='red',
    popup='My Location!',
    tooltip='Circle tooltip'
).add_to(mymap)

mcg = folium.plugins.MarkerCluster(control=False)
mymap.add_child(mcg)

public = folium.plugins.FeatureGroupSubGroup(mcg, "public")
mymap.add_child(public)

privacy = folium.plugins.FeatureGroupSubGroup(mcg, "privacy")
mymap.add_child(privacy)

folium.Marker([37.5718747899, 126.9794814658], icon=folium.Icon(color='pink')).add_to(public)
folium.Marker([37.5795440102, 126.9672918715], icon=folium.Icon(color='pink')).add_to(public)

folium.Marker([37.6023537218, 126.9589709966], icon=folium.Icon(color='green')).add_to(privacy)
folium.Marker([37.5717063998, 127.007613355], icon=folium.Icon(color='green')).add_to(privacy)

folium.LayerControl(collapsed=False, position='bottomright').add_to(mymap)
#width가 안먹히네요,,,ㅠㅠ
plugins.Geocoder(position='bottomright', width='93px').add_to(mymap)


mymap


[37.6102, 127.0071]


In [4]:
class MappingByCoord:

    
    def __init__(self, df, lat, lng, dist = 5):

        self.df = df
        self.lat = lat
        self.lng = lng
        self.dist = dist
        
    def filter_by_rectangle(self):
        """
        사각 범위 내 데이터 필터링
        """
        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist

        lng_min = self.lng - 0.015 * self.dist
        lng_max = self.lng + 0.015 * self.dist

        self.points = [[lat_min, lng_min], [lat_max, lng_max]]

        result = self.df.loc[
            (self.df['lat'] > lat_min) &
            (self.df['lat'] < lat_max) &
            (self.df['lng'] > lng_min) &
            (self.df['lng'] < lng_max)
        ]
        result.index = range(len(result))

        return result    
        

    def filter_by_radius(self):

        tmp = self.filter_by_rectangle()

        center = (self.lat, self.lng)

        result = pd.DataFrame()

        for index, row in tmp.iterrows():
            # 개별 좌표 포인트
            point = (row['lat'], row['lng'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, tmp.iloc[index, :].to_frame().T])
            
        result.index = range(len(result))
            
        return result
        
    def plot_by_rectangle(self, df):
        """
        사각 범위 내 데이터 플로팅
        """

        m = folium.Map(location=[self.lat, self.lng], zoom_start=14)
        

        for idx, row in df.iterrows():

            lat_ = row['lat']
            lng_ = row['lng']

            folium.Marker(location=[lat_, lng_],
                          radius=15,
                          tooltip=row['기관명']).add_to(m)

        folium.Rectangle(bounds=self.points,
                         color='#ff7800',
                         fill=True,
                         fill_color='#ffff00',
                         fill_opacity=0.2).add_to(m)

        return m 
    
    def plot_by_radius(self, df):
        """
        반경 범위 내 데이터 플로팅
        """

        m = folium.Map(location=[self.lat, self.lng], zoom_start=14)

        for idx, row in df.iterrows():

            lat_ = row['lat']
            lng_ = row['lng']

            folium.Marker(location=[lat_, lng_],
                          radius=15,
                          tooltip=row['기관명']).add_to(m)

        folium.Circle(radius=dist * 1000,
                      location=[lat, lng],
                      color="#ff7800",
                      fill_color='#ffff00',
                      fill_opacity=0.2
                      ).add_to(m)

        return m

In [5]:
user_loc = geocoder.ip('me')

mylocation = user_loc.latlng

lat = mylocation[0]
lng = mylocation[1]
dist = 5


mbc = MappingByCoord(df, lat, lng, dist)



In [69]:
result_rectangle = mbc.filter_by_rectangle()

result_radius = mbc.filter_by_radius()

print(f"""
{"="*50}
중심 위도: {mbc.lat}
중심 경도: {mbc.lng}
기준 거리: {mbc.dist} km
사각 범위 내 데이터 필터링 결과: {len(result_rectangle):,} 건
반경 범위 내 데이터 필터링 결과: {len(result_radius):,} 건
{"="*50}
""")


중심 위도: 37.6102
중심 경도: 127.0071
기준 거리: 5 km
사각 범위 내 데이터 필터링 결과: 241 건
반경 범위 내 데이터 필터링 결과: 121 건



In [70]:
plot_1 = mbc.plot_by_rectangle(result_rectangle)
plot_1

In [71]:
plot_2 = mbc.plot_by_radius(result_radius)
plot_2

In [73]:
with open('./전국_정신건강관련기관_위치정보.json', 'r',encoding='utf-8') as f:
    loc_data = json.load(f)
    
loc_arr = loc_data['data']
loc_arr

df = pd.DataFrame(loc_arr)


df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')


df.index = range(len(df))
df

,index,공공/민간,기관구분,기관명,주소,위도,경도,전화번호,홈페이지
0,0,공공,상담소,(주)마음꼭 차심리상담센터,경기도 성남시 분당구 야탑동 351 기억력센터 2층 (야탑로 59 분당차병원),37.410250,127.125250,031-780-5437,None
1,1,공공,상담소,(주)마음연구소 부설 마음iN심리상담센터,서울 종로구 종로3길 34 6층,37.571875,126.979481,02-722-2883,None
2,2,공공,상담소,가능성연구소,부산시 해운대구 세실로79 부산해운대대교사옥 6층,35.171647,129.176440,051-704-8610,None
3,3,공공,상담소,가톨릭대학교 은평성모병원,서울 은평구 진관동 93-6,37.633251,126.916547,02-1811-7755,None
4,4,공공,상담소,강원대학교병원 정신건강의학과,강원도 춘천시 백령로 156 강원대학교병원 정신건강의학과,37.874701,127.744801,033-258-2437,None
...,...,...,...,...,...,...,...,...,...
4327,4706,민간,병원,실버한방병원,서울특별시 금천구 독산로 362 (독산동),37.479089,126.904289,02-857-2001,None
4328,4707,민간,병원,의료법인 영명의료재단 혜민한방병원,"경기도 시흥시 군자로 526 도일그랜드프라자 302,401,501호 (거모동)",37.343354,126.785183,031-504-7272,None
4329,4708,민간,병원,한도한방병원,경기도 의정부시 입석로 36 (가능동),37.755883,127.021652,031-877-6081,None
4330,4709,민간,병원,혜민한방병원,충청남도 태안군 태안읍 정주내1길 13-0 혜민한방병원,36.745075,126.304532,041-400-7171,None


In [ ]:
class MappingByCoord:

    
    def __init__(self, df, lat, lng, dist = 5):

        self.df = df
        self.lat = lat
        self.lng = lng
        self.dist = dist
        
    def setRectangler(self):

        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist

        lng_min = self.lng - 0.015 * self.dist
        lng_max = self.lng + 0.015 * self.dist

        self.points = [[lat_min, lng_min], [lat_max, lng_max]]

        result = self.df.loc[
            (self.df['위도'] > lat_min) &
            (self.df['위도'] < lat_max) &
            (self.df['경도'] > lng_min) &
            (self.df['경도'] < lng_max)
        ]
        result.index = range(len(result))

        return result    
        

    def setCircle(self):

        tmp = self.setRectangler()

        center = (self.lat, self.lng)

        result = pd.DataFrame()

        for index, row in tmp.iterrows():
            point = (row['위도'], row['경도'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, tmp.iloc[index, :].to_frame().T])
            
        result.index = range(len(result))
            
        return result
        
    def MappingInRectangler(self, df):


        m = folium.Map(location=[self.lat, self.lng], zoom_start=14)
        

        for idx, row in df.iterrows():

            lat_now = row['위도']
            lng_now = row['경도']

            folium.Marker(location=[lat_now, lng_now],
                          radius=15,
                          tooltip=row['기관명']).add_to(m)

        folium.Rectangle(bounds=self.points,
                         color='#ff7800',
                         fill=True,
                         fill_color='#ffff00',
                         fill_opacity=0.2).add_to(m)

        return m 
    
    def MappingInCircle(self, df):

        m = folium.Map(location=[self.lat, self.lng], zoom_start=14)

        for idx, row in df.iterrows():

            lat_now = row['위도']
            lng_now = row['경도']

            folium.Marker(location=[lat_now, lng_now],
                          radius=15,
                          tooltip=row['기관명']).add_to(m)

        folium.Circle(radius=dist * 1000,
                      location=[lat, lng],
                      color="#ff7800",
                      fill_color='#ffff00',
                      fill_opacity=0.2
                      ).add_to(m)

        return m